In [7]:
import random
import pandas as pd
import matplotlib.pyplot as plt

# Driver names, team and skill level (How higher they are likely to finish)
drivers = [
    {"name":"Lewis Hamilton", "team": "Mercedes", "skill": 98},
    {"name":"Max Verstappen", "team": "Red Bull Racing", "skill": 99},
    {"name":"George Rusell", "team": "Mercedes", "skill": 85},
    {"name":"Sergio Perez", "team": "Red Bull Racing", "skill": 80},
    {"name":"Charles Leclerc", "team": "Ferrari", "skill": 93},
    {"name":"Carlos Sainz", "team": "Ferrari", "skill": 85},
    {"name":"Lando Norris", "team": "McLaren", "skill": 85},
    {"name":"Oscar Piastri", "team": "McLaren", "skill": 75},
    {"name":"Fernando Alonso", "team": "Aston Martin", "skill": 95},
    {"name":"Esteban Ocon", "team": "Alpine", "skill": 70},
    {"name":"Lance Stroll", "team": "Aston Martin", "skill": 70},
    {"name":"Pierre Gasly", "team": "Alpine", "skill": 65},
    {"name":"Logan Sargeant", "team": "Williams", "skill": 60},
    {"name":"Alexander Albon", "team": "Williams", "skill": 65},
    {"name":"Kevin Magnussen", "team": "Haas", "skill": 75},
    {"name":"Nico Hulkenberg", "team": "Haas", "skill": 75},
    {"name":"Nyck De Vries", "team": "AlphaTauri", "skill": 65},
    {"name":"Yuki Tsunoda", "team": "AlphaTauri", "skill": 55},
    {"name":"Valtteri Bottas", "team": "Alfa Romeo", "skill": 75},
    {"name":"Zhou Guanyu", "team": "Alfa Romeo", "skill": 60}
]

# Team names, car performance and reliability (How likely the car is to finish a race)
teams = [
    {"name":"Mercedes", "performance": 85, "reliability": 0.90},
    {"name":"Red Bull Racing", "performance": 97, "reliability": 0.95},
    {"name":"Ferrari", "performance": 80, "reliability": 0.75},
    {"name":"McLaren", "performance": 65, "reliability": 0.70},
    {"name":"Alpine", "performance": 77, "reliability": 0.75},
    {"name":"Aston Martin", "performance": 85, "reliability": 0.90},
    {"name":"Williams", "performance": 70, "reliability": 0.85},
    {"name":"Haas", "performance": 65, "reliability": 0.76},
    {"name":"AlphaTauri", "performance": 55, "reliability": 0.83},
    {"name":"Alfa Romeo", "performance": 75, "reliability": 0.79}
]


f1_calendar = [
    "Bahrain Grand Prix",
    "Saudi Arabian Grand Prix",
    "Australia Grand Prix",
    "Azerbaijan Grand Prix",
    "Miami Grand Prix",
    "Emilia Romagna Grand Prix",
    "Monaco Grand Prix",
    "Spanish Grand Prix",
    "Canadian Grand Prix",
    "Austrian Grand Prix",
    "British Grand Prix",
    "Hungarian Grand Prix",
    "Belgian Grand Prix",
    "German Grand Prix",
    "Dutch Grand Prix",
    "Italian Grand Prix",
    "Singapore Grand Prix",
    "Japanese Grand Prix",
    "Qatar Grand Prix",
    "United States Grand Prix",
    "Mexican Grand Prix",
    "Brazilian Grand Prix",
    "Las Vegas Grand Prix",
    "Abu Dhabi Grand Prix"
]


driver_points = {}
driver_wins = {}
driver_positions = {}
driver_podiums = {}
for driver in drivers:
    driver_points[driver["name"]] = 0
    driver_wins[driver["name"]] = 0
    driver_positions[driver["name"]] = []
    driver_podiums[driver["name"]] = 0

# Simulates the results of a single race with provided driver and teams
def create_race_results(drivers, teams):
    results = {}

    # Factor in driver skill, car performance and reliability in the race simulation
    for driver in drivers:
        driver_performance = driver["skill"]
        car_performance = [t["performance"] for t in teams if t["name"] == driver["team"]][0]
        result = random.randint(1, 10) + (car_performance / 10) + (driver_performance / 10)
        reliability = [t["reliability"] for t in teams if t["name"] == driver["team"]][0]

        if random.random() > reliability:
            result = 0
        results[driver["name"]] = result

    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

    for i, result in enumerate(sorted_results):
        driver = result[0]
        points = 0
        position = i + 1
        if result[1] == 0:
            driver_positions[driver].append(0)
            continue

        driver_positions[driver].append(position)

        if position == 1:
            points = 25
            driver_wins[driver] += 1
            driver_podiums[driver] += 1
        elif position == 2:
            points = 18
            driver_podiums[driver] += 1
        elif position == 3:
            points = 15
            driver_podiums[driver] += 1
        elif position == 4:
            points = 12
        elif position == 5:
            points = 10
        elif position == 6:
            points = 8
        elif position == 7:
            points = 6
        elif position == 8:
            points = 4
        elif position == 9:
            points = 2
        elif position == 10:
            points = 1
        driver_points[driver] += points

    return driver_points, driver_wins, driver_positions, driver_podiums

results = []

# Iterate through the f1 calendar and simulate races
for i in range(len(f1_calendar)):
    driver_points, driver_wins, driver_positions, driver_podiums = create_race_results(drivers, teams)
    import json
    json.dumps(driver_positions)


# Extract driver details (name, team, points, wins, podiums, positions) and append to results list.
for driver in driver_points:
    name = driver
    team = [d["team"] for d in drivers if d["name"] == name][0]
    points = driver_points[name]
    wins = driver_wins[name]
    podiums = driver_podiums[name]
    positions = driver_positions[name]
    results.append([name, team, points, wins, podiums] + positions)

# Create a table of the season's results
df = pd.DataFrame(results, columns=["Driver", "Team", "Points", "Wins", "Podiums"] + [f"{f1_calendar[i]}" for i in range(24)])

df = df.sort_values(by=["Points", "Wins", "Podiums"], ascending=False)

# Colour the podium finishers
def color_win(val):
    if val == '0':
        return 'background-color: purple'
    elif isinstance(val, int):
        if val == 1:
            return 'background-color: gold'
        elif val == 2:
            return 'background-color: grey'
        elif val == 3:
            return 'background-color: brown'
        elif val <= 10 and val != 0:
            return 'background-color: green'
    return ''

dfs = df.style.applymap(color_win, subset=[f"{f1_calendar[i]}" for i in range(len(f1_calendar))])

constructor_points = {}
constructor_wins = {}
constructor_positions = {}
constructor_podiums = {}

for driver in drivers:
    team = driver["team"]
    constructor_points[team] = constructor_points.get(team, 0) + driver_points[driver["name"]]
    constructor_wins[team] = constructor_wins.get(team, 0) + driver_wins[driver["name"]]
    constructor_podiums[team] = constructor_podiums.get(team, 0) + driver_podiums[driver["name"]]
    constructor_positions[team] = constructor_positions.get(team, [])
    constructor_positions[team].extend(positions for d, positions in zip(drivers, driver_positions.values()) if d["team"] == team)

results = []

for team in constructor_points:
    drivers_team = [d for d in drivers if d["team"] == team]
    drivers_table = pd.DataFrame(index=[d["name"] for d in drivers_team], columns=[f"{f1_calendar[i]}" for i in range(len(f1_calendar))])
    for driver in drivers_team:
        drivers_table.loc[driver["name"]] = driver_positions[driver["name"]]
    results.append([team, constructor_points[team], constructor_wins[team], constructor_podiums[team]])

# Create a table of the constructors' results
df_constructors = pd.DataFrame(results, columns=["Team", "Points", "Wins", "Podiums"])

df_constructors = df_constructors.sort_values(by=["Points", "Wins", "Podiums"], ascending=False)

def color_win(val):
    color = "yellow" if val == 25 else ("grey" if val == 18 else ("brown" if val == 15 else None))
    return f"background-color: {color}" if color else ""

def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

df_constructors = df_constructors.style.applymap(color_win, subset=["Wins"]).apply(highlight_max, subset=["Points", "Wins", "Podiums"])

display(dfs)
display(df_constructors)


,Driver,Team,Points,Wins,Podiums,Bahrain Grand Prix,Saudi Arabian Grand Prix,Australia Grand Prix,Azerbaijan Grand Prix,Miami Grand Prix,Emilia Romagna Grand Prix,Monaco Grand Prix,Spanish Grand Prix,Canadian Grand Prix,Austrian Grand Prix,British Grand Prix,Hungarian Grand Prix,Belgian Grand Prix,German Grand Prix,Dutch Grand Prix,Italian Grand Prix,Singapore Grand Prix,Japanese Grand Prix,Qatar Grand Prix,United States Grand Prix,Mexican Grand Prix,Brazilian Grand Prix,Las Vegas Grand Prix,Abu Dhabi Grand Prix
1,Max Verstappen,Red Bull Racing,310,8,11,2,7,4,0,1,1,0,1,1,5,9,1,1,6,3,1,1,8,0,5,3,5,0,0
3,Sergio Perez,Red Bull Racing,303,5,11,7,5,1,1,10,4,3,0,11,3,3,5,5,1,2,6,0,7,1,3,5,3,1,4
0,Lewis Hamilton,Mercedes,284,4,11,1,6,8,3,7,3,10,5,2,1,5,3,2,11,9,0,5,4,3,1,7,2,10,1
8,Fernando Alonso,Aston Martin,268,3,9,5,1,2,10,2,5,6,6,10,2,0,4,4,9,10,8,9,5,5,2,1,1,3,3
4,Charles Leclerc,Ferrari,260,3,11,14,2,5,2,0,2,2,7,3,12,1,2,3,2,1,5,0,1,10,11,0,0,4,6
2,George Rusell,Mercedes,188,0,7,3,14,10,4,0,10,12,3,13,7,6,9,0,4,5,2,3,3,4,6,2,13,2,9
18,Valtteri Bottas,Alfa Romeo,162,1,2,6,9,3,8,14,6,1,16,4,4,13,6,6,5,12,0,8,0,7,14,4,4,5,7
5,Carlos Sainz,Ferrari,119,0,5,0,12,7,5,3,0,8,14,9,0,0,0,10,0,13,3,2,11,2,8,6,12,11,2
10,Lance Stroll,Aston Martin,104,0,2,8,3,15,6,5,13,4,15,5,0,2,10,11,14,4,7,6,14,0,16,12,11,0,13
9,Esteban Ocon,Alpine,98,0,0,13,10,18,13,4,0,0,10,15,8,4,7,7,13,7,4,4,13,8,15,8,0,6,5


,Team,Points,Wins,Podiums
1,Red Bull Racing,613,13,22
0,Mercedes,472,4,18
2,Ferrari,379,3,16
4,Aston Martin,372,3,11
9,Alfa Romeo,184,1,2
5,Alpine,147,0,1
3,McLaren,107,0,1
6,Williams,75,0,0
7,Haas,45,0,1
8,AlphaTauri,30,0,0
